# Big Data, Programa de Ciencia de los Datos
## Tarea #1

* Esteban Sáenz Villalobos (**esaenz7@gmail.com**)
* Entrega: 08 de agosto 2021, 22:00.
* Observaciones: Trabajo elaborado desde Google Colab. Ejecutar cada celda de código de forma secuencial.

---

In [1]:
from IPython.display import Javascript

In [2]:
'''
Instalación de PySpark en Colab
'''
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import requests, os
from bs4 import BeautifulSoup
#obtener las versiones de spark e instalar la última disponile
soup = BeautifulSoup(requests.get('https://downloads.apache.org/spark/').text)
link_files = []
[link_files.append(link.get('href')) for link in soup.find_all('a')]
spark_link = [x for x in link_files if 'spark' in x]
ver_spark = spark_link[-1][:-1]
os.system(f"wget -q https://www-us.apache.org/dist/spark/{ver_spark}/{ver_spark}-bin-hadoop3.2.tgz")
os.system(f"tar xf {ver_spark}-bin-hadoop2.7.tgz")
#instalar pyspark
!pip install -q pyspark
!pip --version
!pyspark --version
#instalar pytests
!pip install -q pytest pytest-sugar
!pytest --version
# from pathlib import Path
# if Path.cwd().name != 'tdd':
#     %mkdir tdd
#     %cd tdd
# %pwd
# %rm *.py
#---

     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 198 kB 54.3 MB/s 
pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/
                        
Using Scala version 2.12.10, OpenJDK 64-Bit Server VM, 11.0.11
Branch HEAD
Compiled by user centos on 2021-05-24T04:27:48Z
Revision de351e30a90dd988b133b3d00fa6218bfcaba8b8
Url https://github.com/apache/spark
Type --help for more information.
This is pytest version 3.6.4, imported from /usr/local/lib/python2.7/dist-packages/pytest.pyc


In [3]:
'''
https://drive.google.com/file/d/14z1rwq2RdsAmWKG02Rkty6e3ntzatRv4/view?usp=sharing
https://drive.google.com/file/d/1MNrkOUEKqdttNOnPCXJK6hxo2lxVZWjG/view?usp=sharing
https://drive.google.com/file/d/1BRXIsAnamKOlTImuMmU-5xBK6uQgyVXK/view?usp=sharing
'''
# import io
# from google.colab import files
# uploaded1 = files.upload()
# uploaded2 = files.upload()
# uploaded3 = files.upload()
!pip install -q gdown
!gdown https://drive.google.com/uc?id=14z1rwq2RdsAmWKG02Rkty6e3ntzatRv4
!gdown https://drive.google.com/uc?id=1MNrkOUEKqdttNOnPCXJK6hxo2lxVZWjG
!gdown https://drive.google.com/uc?id=1BRXIsAnamKOlTImuMmU-5xBK6uQgyVXK

Downloading...
From: https://drive.google.com/uc?id=14z1rwq2RdsAmWKG02Rkty6e3ntzatRv4
To: /content/ciclista.csv
100% 1.82k/1.82k [00:00<00:00, 3.83MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MNrkOUEKqdttNOnPCXJK6hxo2lxVZWjG
To: /content/ruta.csv
100% 298/298 [00:00<00:00, 245kB/s]
Downloading...
From: https://drive.google.com/uc?id=1BRXIsAnamKOlTImuMmU-5xBK6uQgyVXK
To: /content/actividad.csv
100% 8.40k/8.40k [00:00<00:00, 24.4MB/s]


In [4]:
%%file procesamientodatos.py
'''
Nombre de archivo:
  +procesamientodatos.py
Descripción: 
  +Librería con funciones para el procesamiento de los datos
Métodos:
  +cargar_datos
  +unir_datos
  +agregar_datos
  +presentar_datos
  +almacenar_datos
'''

#librerías necesarias
import sys, os, datetime
from pyspark.sql import SparkSession, functions as F, window as W
from pyspark.sql.types import (DateType, IntegerType, FloatType, DoubleType, StringType, StructField, StructType, TimestampType)

#sesión de spark
spark = SparkSession.builder\
        .master("local")\
        .appName("App#1")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

#función para carga de datos (lista con archivos csv)
def cargar_datos(files=[], show=20, print_=True):
  try:
    #lectura de archivos a partir de la definición de esquemas
    df1 = spark.read.csv(files[1], schema=StructType(\
                                  [StructField('cedula', IntegerType()),
                                  StructField('nombre', StringType()),
                                  StructField('provincia', StringType()),]))
    df2 = spark.read.csv(files[2], schema=StructType(\
                                  [StructField('codigo_ruta', IntegerType()),
                                  StructField('nombre_ruta', StringType()),
                                  StructField('kms', FloatType()),]))
    df3 = spark.read.csv(files[3], schema=StructType(\
                                  [StructField('codigo_ruta', IntegerType()),
                                  StructField('cedula', IntegerType()),
                                  StructField('fecha', DateType()),]))
    #impresión de resultados
    if print_:
      print('DataFrame1')
      df1.show(show)
      df1.printSchema()
      print('DataFrame2')
      df2.show(show)
      df2.printSchema()
      print('DataFrame3')
      df3.show(show)
      df3.printSchema()
    return [df1, df2, df3]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para unión de datos de los dataframes (data=lista de dataframes) y selección de las columnas requeridas (select=lista de columnas)
def unir_datos(data=[], select=[], show=20, print_=True):
  try:
    #unión de los dataframes a partir de las columnas relacionadas
    dfResultados1 = data[2].join(data[1], data[1].codigo_ruta == data[2].codigo_ruta)\
    .join(data[0], data[0].cedula == data[2].cedula)
    #selección de las columnas requeridas
    dfResultados2 = dfResultados1.select(select).dropna()
    #impresión de resultados
    if print_:
      dfResultados1.show(show)
      dfResultados2.show(show)
    return [dfResultados1, dfResultados2]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para agrupar y agregar los datos (data=dataframe) a partir de las columnas especificadas (group=columnas a agrupar, agg=columna de agregación)
def agregar_datos(data=[], group=[], agg='', show=20, print_=True):
  try:
    #agrupación y agregación de los datos (totales y promedios)
    dfResultados3 = data[1].groupBy(group).agg(F.sum(agg),F.mean(agg))\
    .withColumn('sum('+agg+')', F.round('sum('+agg+')',2))\
    .withColumn('avg('+agg+')', F.round('avg('+agg+')',2))
    #impresión de resultados
    if print_:
      dfResultados3.show(show)
    return [dfResultados3]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para presentar los datos (data=dataframe) particionados (part=columna) y ordenados (order=columna) con un límite (top=cantidad de filas)
def presentar_datos(data=[], top=5, part='', order='', show=20, print_=True):
  try:
    #definición de operación de partición y ordenamiento
    window = W.Window.partitionBy(part).orderBy(F.col(part).desc(), F.col(order).desc())
    #partición y ordenamiento de los datos
    dfResultados4 = data[0].withColumn('row',F.row_number().over(window))\
    .filter(F.col('row')<=top).drop('row')
    #impresión de resultados
    if print_:
      print('\nDataFrame: Top 5 total de kms y promedio de kms diario, por provincia.')
      dfResultados4.show(show)
      print('\nEsquema del dataframe.')
      dfResultados4.printSchema()
      print('\nExplicación de ejecución de spark.')
      dfResultados4.explain()
    return [dfResultados4]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

#función para guardar los datos (data=dataframe) con nombre (nombre=nombre del archivo)
def almacenar_datos(data=[], nombre='default.csv', show=20, print_=True):
  try:
    #escritura de archivo
    data[0].write.csv(nombre, mode='overwrite')
    #lectura de archivo guardado
    dfResultados5 = spark.read.csv(nombre, schema=data[0].schema)
    #impresión de resultados
    if print_:
      print('\nDataFrame: obtenido del archivo '+nombre+'.')
      dfResultados5.show(show)
      print('\nDataFrame: descripción de los datos por provincia.')
      dfResultados5.groupby(dfResultados5[0]).agg(F.count(dfResultados5[2]).alias('count'),F.min(dfResultados5[2]).alias('min'),F.max(dfResultados5[2]).alias('max'),F.round(F.mean(dfResultados5[2]),2).alias('avg')).show()
    return [dfResultados5]
  except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print(exc_type, os.path.split(exc_tb.tb_frame.f_code.co_filename)[1], exc_tb.tb_lineno, exc_obj)

Writing procesamientodatos.py


In [5]:
%%file programaestudiante.py
'''
Nombre de archivo:
  +programaestudiante.py
Descripción: 
  +Archivo principal (main) para la ejecución del programa
'''

#librerías necesarias
import sys, os
from procesamientodatos import *

#llamado a las diferentes funciones para el procesamiento de los datos por etapas (stage#)
stage1 = cargar_datos(sys.argv)
stage2 = unir_datos(stage1, select=['fecha','nombre','provincia','nombre_ruta','kms'])
stage3 = agregar_datos(stage2, group=['provincia','nombre'], agg='kms')
stage4 = presentar_datos(stage3, top=5, part='provincia', order='sum(kms)', show=50)
stage5 = almacenar_datos(stage4, nombre='resultados.csv', show=50)

Writing programaestudiante.py


In [6]:
%%file conftest.py
'''
Nombre de archivo:
  +conftest.py
Descripción: 
  +Archivo para definición del contexto para la ejecución de las pruebas
'''

#librerías necesarias
import pytest
from procesamientodatos import *

#sesión de spark
@pytest.fixture(scope="module")
def spark_session():
  spark = SparkSession.builder\
        .master("local")\
        .appName("Test#1")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
  spark.sparkContext.setLogLevel("ERROR")
  yield spark
  spark.stop()

#definición de los parámetros (fixtures) de pruebas según las diferentes etapas de ejecución del programa (stages)
#
#***resultados actuales*** obtenidos durante la ejecución del programa de forma regular
#
#parámetro para prueba de carga de datos
stage1 = cargar_datos(['','ciclista.csv','ruta.csv','actividad.csv'])
@pytest.fixture
def tstage1():
  return stage1
#parámetro para prueba de unión de datos
stage2 = unir_datos(stage1, select=['fecha','nombre','provincia','nombre_ruta','kms'])
@pytest.fixture
def tstage2():
  return stage2
#parámetro para prueba de agregaciones parciales
stage3 = agregar_datos(stage2, group=['provincia','nombre'], agg='kms')
@pytest.fixture
def tstage3():
  return stage3
#parámetro para prueba de resultados finales
stage4 = presentar_datos(stage3, top=5, part='provincia', order='sum(kms)')
@pytest.fixture
def tstage4():
  return stage4
#parámetro para prueba de almacenamiento
stage5 = almacenar_datos(stage4, nombre='resultados.csv')
@pytest.fixture
def tstage5():
  return stage5
#
#***resultados actuales*** obtenidos durante la ejecución del programa alterando valores en los datos
#
#parámetro para prueba completa con valores en cero
@pytest.fixture
def tzerodata():
  dfzerodata = cargar_datos(['','ciclista.csv','ruta.csv','actividad.csv'])
  dfzerodata[1] = dfzerodata[1].withColumn('kms', F.when(dfzerodata[1]['kms'] < 40, 0).otherwise(dfzerodata[1]['kms'])) #se sustituyen algunos valores por 0 (valores menores a 40km se reemplazan por 0)
  rzerodata = presentar_datos(agregar_datos(unir_datos(dfzerodata, select=['fecha','nombre','provincia','nombre_ruta','kms']), group=['provincia','nombre'], agg='kms'), top=5, part='provincia', order='sum(kms)')
  return rzerodata
#parámetro para prueba completa con valores numéricos faltantes
@pytest.fixture
def tmissnumdata():
  dfmissnumdata = cargar_datos(['','ciclista.csv','ruta.csv','actividad.csv'])
  dfmissnumdata[1] = dfmissnumdata[1].withColumn('kms', F.when(dfmissnumdata[1]['kms'] < 40, '').otherwise(dfmissnumdata[1]['kms'])) #se sustituyen algunos valores numéricos por nulos (valores menores a 40km se reemplazan por valor nulo)
  rmissnumdata = presentar_datos(agregar_datos(unir_datos(dfmissnumdata, select=['fecha','nombre','provincia','nombre_ruta','kms']), group=['provincia','nombre'], agg='kms'), top=5, part='provincia', order='sum(kms)')
  return rmissnumdata
#parámetro para prueba completa con valores categóricos faltantes
@pytest.fixture
def tmisscatdata():
  dfmisscatdata = cargar_datos(['','ciclista.csv','ruta.csv','actividad.csv'])
  dfmisscatdata[0] = dfmisscatdata[0].withColumn('provincia', F.when(dfmisscatdata[0]['provincia'] == 'Alajuela', '').otherwise(dfmisscatdata[0]['provincia'])) #se sustituyen algunos valores categóricos por nulos (valores de columna provincia se reemplazan por valor nulo)
  rmisscatdata = presentar_datos(agregar_datos(unir_datos(dfmisscatdata, select=['fecha','nombre','provincia','nombre_ruta','kms']), group=['provincia','nombre'], agg='kms'), top=5, part='provincia', order='sum(kms)')
  return rmisscatdata

#***resultados esperados*** para pruebas de comparación
#
#parámetro para prueba de resultados finales
@pytest.fixture
def tstage4_expected():
  data = [('San José', 'FERNANDEZ ANDUJAR Ruben', 605.96, 43.28),
          ('San José', 'ALAPHILIPPE Julian', 348.77, 49.82),
          ('San José', 'ROJAS José Joaquin', 337.85, 37.54),
          ('San José', 'KÄMNA Lennard', 259.72, 51.94),
          ('San José', 'MOSCON Gianni', 253.49, 50.7),
          ('Heredia', 'CARAPAZ RICHARD ANTONIO', 434.51, 48.28),
          ('Heredia', 'PUCCIO Salvatore', 420.38, 38.22),
          ('Heredia', 'KELDERMAN Wilco', 390.18, 39.02),
          ('Heredia', 'MARCZYNSKI Tomasz', 361.53, 40.17),
          ('Heredia', 'NIELSEN Magnus Cort', 319.21, 45.6),
          ('Alajuela', 'WALLAYS Jelle', 517.33, 47.03),
          ('Alajuela', 'CHAVES RUBIO Johan Esteban', 413.35, 41.34),
          ('Alajuela', 'POLJANSKI Pawel', 355.0, 32.27),
          ('Alajuela', 'BARGUIL WARREN', 235.91, 39.32),
          ('Alajuela', 'ROCHE Nicolas', 207.31, 41.46),
          ('Limón', 'NIBALI Vincenzo', 239.48, 39.91),
          ('Limón', 'KONRAD Patrick', 205.88, 51.47),
          ('Limón', 'DE CLERCQ Bart', 204.79, 40.96),
          ('Limón', 'FROOME Christopher', 89.22, 44.61),
          ('Limón', 'MAJKA Rafal', 71.43, 35.72),
          ('Cartago', 'POELS Wout', 403.7, 40.37),
          ('Cartago', 'OSS Daniel', 379.12, 47.39),
          ('Cartago', 'BENEDETTI Cesare', 247.72, 41.29),
          ('Cartago', 'LAMPAERT Yves', 246.85, 41.14),
          ('Cartago', 'HAGA Chad', 163.24, 40.81),
          ('Guanacaste', 'YATES Simon', 440.34, 44.03),
          ('Guanacaste', 'KNEES Christian', 440.15, 48.91),
          ('Guanacaste', 'STANNARD Ian', 315.28, 45.04),
          ('Guanacaste', 'DENNIS Rohan', 241.37, 48.27),
          ('Puntarenas', 'BETANCUR GOMEZ Carlos Alberto', 460.15, 38.35),
          ('Puntarenas', 'HAIG Jack', 222.92, 55.73),
          ('Puntarenas', 'BUCHMANN Emanuel', 185.87, 37.17)]
  schema = StructType(\
                      [StructField('provincia',StringType()),
                      StructField('nombre',StringType()),
                      StructField('sum(kms)',DoubleType()),
                      StructField('avg(kms)',DoubleType()),])
  stage4_expected = spark.createDataFrame(data, schema)
  return stage4_expected
#parámetro para prueba completa con valores en cero
@pytest.fixture
def tzerodata_expected():
  data = [('San José', 'FERNANDEZ ANDUJAR Ruben', 428.93, 30.64),
          ('San José', 'ALAPHILIPPE Julian', 282.86, 40.41),
          ('San José', 'ROJAS José Joaquin', 240.15, 26.68),
          ('San José', 'PEDRERO Antonio', 231.82, 46.36),
          ('San José', 'SOLER GIMENEZ Marc', 196.04, 39.21),
          ('Puntarenas', 'BETANCUR GOMEZ Carlos Alberto', 356.21, 29.68),
          ('Puntarenas', 'HAIG Jack', 183.35, 45.84),
          ('Puntarenas', 'BUCHMANN Emanuel', 102.84, 20.57),
          ('Alajuela', 'WALLAYS Jelle', 445.13, 40.47),
          ('Alajuela', 'CHAVES RUBIO Johan Esteban', 308.93, 30.89),
          ('Alajuela', 'POLJANSKI Pawel', 254.64, 23.15),
          ('Alajuela', 'ROCHE Nicolas', 150.62, 30.12),
          ('Alajuela', 'BARGUIL WARREN', 107.76, 17.96),
          ('Limón', 'NIBALI Vincenzo', 206.37, 34.4),
          ('Limón', 'KONRAD Patrick', 205.88, 51.47),
          ('Limón', 'DE CLERCQ Bart', 149.23, 29.85),
          ('Limón', 'MAJKA Rafal', 53.88, 26.94),
          ('Limón', 'FROOME Christopher', 49.65, 24.83),
          ('Heredia', 'CARAPAZ RICHARD ANTONIO', 359.21, 39.91),
          ('Heredia', 'PUCCIO Salvatore', 335.96, 30.54),
          ('Heredia', 'KELDERMAN Wilco', 269.55, 26.96),
          ('Heredia', 'NIELSEN Magnus Cort', 230.63, 32.95),
          ('Heredia', 'MARCZYNSKI Tomasz', 230.34, 25.59),
          ('Cartago', 'POELS Wout', 351.48, 35.15),
          ('Cartago', 'OSS Daniel', 331.39, 41.42),
          ('Cartago', 'BENEDETTI Cesare', 183.35, 30.56),
          ('Cartago', 'HAGA Chad', 145.69, 36.42),
          ('Cartago', 'LAMPAERT Yves', 134.6, 22.43),
          ('Guanacaste', 'YATES Simon', 393.04, 39.3),
          ('Guanacaste', 'KNEES Christian', 386.87, 42.99),
          ('Guanacaste', 'STANNARD Ian', 282.17, 40.31),
          ('Guanacaste', 'DENNIS Rohan', 184.25, 36.85)]
  schema = StructType(\
                      [StructField('provincia',StringType()),
                      StructField('nombre',StringType()),
                      StructField('sum(kms)',DoubleType()),
                      StructField('avg(kms)',DoubleType()),])
  result = spark.createDataFrame(data, schema)
  return result
#parámetro para prueba completa con valores numéricos faltantes
@pytest.fixture
def tmissnumdata_expected():
  data = [('San José', 'FERNANDEZ ANDUJAR Ruben', 428.93, 53.62),
          ('San José', 'ALAPHILIPPE Julian', 282.86, 56.57),
          ('San José', 'ROJAS José Joaquin', 240.15, 60.04),
          ('San José', 'PEDRERO Antonio', 231.82, 57.96),
          ('San José', 'SOLER GIMENEZ Marc', 196.04, 49.01),
          ('Puntarenas', 'BETANCUR GOMEZ Carlos Alberto', 356.21, 50.89),
          ('Puntarenas', 'HAIG Jack', 183.35, 61.12),
          ('Puntarenas', 'BUCHMANN Emanuel', 102.84, 51.42),
          ('Alajuela', 'WALLAYS Jelle', 445.13, 55.64),
          ('Alajuela', 'CHAVES RUBIO Johan Esteban', 308.93, 51.49),
          ('Alajuela', 'POLJANSKI Pawel', 254.64, 50.93),
          ('Alajuela', 'ROCHE Nicolas', 150.62, 50.21),
          ('Alajuela', 'BARGUIL WARREN', 107.76, 53.88),
          ('Limón', 'NIBALI Vincenzo', 206.37, 51.59),
          ('Limón', 'KONRAD Patrick', 205.88, 51.47),
          ('Limón', 'DE CLERCQ Bart', 149.23, 49.74),
          ('Limón', 'MAJKA Rafal', 53.88, 53.88),
          ('Limón', 'FROOME Christopher', 49.65, 49.65),
          ('Heredia', 'CARAPAZ RICHARD ANTONIO', 359.21, 51.32),
          ('Heredia', 'PUCCIO Salvatore', 335.96, 55.99),
          ('Heredia', 'KELDERMAN Wilco', 269.55, 67.39),
          ('Heredia', 'NIELSEN Magnus Cort', 230.63, 57.66),
          ('Heredia', 'MARCZYNSKI Tomasz', 230.34, 57.59),
          ('Cartago', 'POELS Wout', 351.48, 50.21),
          ('Cartago', 'OSS Daniel', 331.39, 55.23),
          ('Cartago', 'BENEDETTI Cesare', 183.35, 61.12),
          ('Cartago', 'HAGA Chad', 145.69, 48.56),
          ('Cartago', 'LAMPAERT Yves', 134.6, 67.3),
          ('Guanacaste', 'YATES Simon', 393.04, 49.13),
          ('Guanacaste', 'KNEES Christian', 386.87, 55.27),
          ('Guanacaste', 'STANNARD Ian', 282.17, 56.43),
          ('Guanacaste', 'DENNIS Rohan', 184.25, 61.42)]
  schema = StructType(\
                      [StructField('provincia',StringType()),
                      StructField('nombre',StringType()),
                      StructField('sum(kms)',DoubleType()),
                      StructField('avg(kms)',DoubleType()),])
  result = spark.createDataFrame(data, schema)
  return result
#parámetro para prueba completa con valores categóricos faltantes
@pytest.fixture
def tmisscatdata_expected():
  data = [('San José', 'FERNANDEZ ANDUJAR Ruben', 605.96, 43.28),
          ('San José', 'ALAPHILIPPE Julian', 348.77, 49.82),
          ('San José', 'ROJAS José Joaquin', 337.85, 37.54),
          ('San José', 'KÄMNA Lennard', 259.72, 51.94),
          ('San José', 'MOSCON Gianni', 253.49, 50.7),
          ('Puntarenas', 'BETANCUR GOMEZ Carlos Alberto', 460.15, 38.35),
          ('Puntarenas', 'HAIG Jack', 222.92, 55.73),
          ('Puntarenas', 'BUCHMANN Emanuel', 185.87, 37.17),
          ('Limón', 'NIBALI Vincenzo', 239.48, 39.91),
          ('Limón', 'KONRAD Patrick', 205.88, 51.47),
          ('Limón', 'DE CLERCQ Bart', 204.79, 40.96),
          ('Limón', 'FROOME Christopher', 89.22, 44.61),
          ('Limón', 'MAJKA Rafal', 71.43, 35.72),
          ('Heredia', 'CARAPAZ RICHARD ANTONIO', 434.51, 48.28),
          ('Heredia', 'PUCCIO Salvatore', 420.38, 38.22),
          ('Heredia', 'KELDERMAN Wilco', 390.18, 39.02),
          ('Heredia', 'MARCZYNSKI Tomasz', 361.53, 40.17),
          ('Heredia', 'NIELSEN Magnus Cort', 319.21, 45.6),
          ('Cartago', 'POELS Wout', 403.7, 40.37),
          ('Cartago', 'OSS Daniel', 379.12, 47.39),
          ('Cartago', 'BENEDETTI Cesare', 247.72, 41.29),
          ('Cartago', 'LAMPAERT Yves', 246.85, 41.14),
          ('Cartago', 'HAGA Chad', 163.24, 40.81),
          ('', 'WALLAYS Jelle', 517.33, 47.03),
          ('', 'CHAVES RUBIO Johan Esteban', 413.35, 41.34),
          ('', 'POLJANSKI Pawel', 355.0, 32.27),
          ('', 'BARGUIL WARREN', 235.91, 39.32),
          ('', 'ROCHE Nicolas', 207.31, 41.46),
          ('Guanacaste', 'YATES Simon', 440.34, 44.03),
          ('Guanacaste', 'KNEES Christian', 440.15, 48.91),
          ('Guanacaste', 'STANNARD Ian', 315.28, 45.04),
          ('Guanacaste', 'DENNIS Rohan', 241.37, 48.27)]
  schema = StructType(\
                      [StructField('provincia',StringType()),
                      StructField('nombre',StringType()),
                      StructField('sum(kms)',DoubleType()),
                      StructField('avg(kms)',DoubleType()),])
  result = spark.createDataFrame(data, schema)
  return result

Writing conftest.py


In [7]:
%%file test_programaestudiante.py
'''
Nombre de archivo:
  +test_programaestudiante.py
Descripción: 
  +Archivo para la ejecuión de las pruebas del programa
'''

#librerías necesarias
import pytest
from pyspark.sql import functions as F

#pruebas según las diferentes etapas de ejecución del programa (stages)
#se comparan los valores actuales obtenidos desde los "fixtures" con los valores esperados según cada condición
#
#prueba de carga de datos
def test_stage1(tstage1):
  tstage1[0].show()
  tstage1[1].show()
  tstage1[2].show()
  assert type(tstage1) == list
  assert len(tstage1) == 3
  assert tstage1[0].count() == 50
  assert tstage1[1].count() == 15
  assert tstage1[2].count() == 300
  assert str(tstage1[0].dtypes) == "[('cedula', 'int'), ('nombre', 'string'), ('provincia', 'string')]"
  assert str(tstage1[1].dtypes) == "[('codigo_ruta', 'int'), ('nombre_ruta', 'string'), ('kms', 'float')]"
  assert str(tstage1[2].dtypes) == "[('codigo_ruta', 'int'), ('cedula', 'int'), ('fecha', 'date')]"
#prueba de unión de datos
def test_stage2(tstage2):
  tstage2[0].show()
  tstage2[1].show()
  assert type(tstage2) == list
  assert len(tstage2) == 2
  assert tstage2[0].count() == 300
  assert tstage2[1].count() == 300
  assert str(tstage2[0].dtypes) == "[('codigo_ruta', 'int'), ('cedula', 'int'), ('fecha', 'date'), ('codigo_ruta', 'int'), ('nombre_ruta', 'string'), ('kms', 'float'), ('cedula', 'int'), ('nombre', 'string'), ('provincia', 'string')]"
  assert str(tstage2[1].dtypes) == "[('fecha', 'date'), ('nombre', 'string'), ('provincia', 'string'), ('nombre_ruta', 'string'), ('kms', 'float')]"
#prueba de agregaciones parciales
def test_stage3(tstage3):
  tstage3[0].show()
  assert type(tstage3) == list
  assert len(tstage3) == 1
  assert tstage3[0].count() == 47
  assert str(tstage3[0].dtypes) == "[('provincia', 'string'), ('nombre', 'string'), ('sum(kms)', 'double'), ('avg(kms)', 'double')]"
#prueba de resultados finales
def test_stage4(tstage4, tstage4_expected):
  tstage4[0].show()
  tstage4_expected.show()
  assert type(tstage4) == list
  assert len(tstage4) == 1
  assert tstage4[0].count() == 32
  assert str(tstage4[0].dtypes) == "[('provincia', 'string'), ('nombre', 'string'), ('sum(kms)', 'double'), ('avg(kms)', 'double')]"
  assert tstage4[0].select('provincia').distinct().count() == 7
  assert str(tstage4[0].select('sum(kms)','avg(kms)').summary("count", "min", "max").collect()) == "[Row(summary='count', sum(kms)='32', avg(kms)='32'), Row(summary='min', sum(kms)='71.43', avg(kms)='32.27'), Row(summary='max', sum(kms)='605.96', avg(kms)='55.73')]"
  assert tstage4_expected.exceptAll(tstage4[0]).count() == 0
  assert tstage4[0].exceptAll(tstage4_expected).count() == 0
  #el método de comparación entre el dataframe esperado y el actual retorna la cantidad de filas distintas entre ambos, por su forma se implementa en ambas vías, actual vs esperado y esperado vs actual
  #es un método simple y funciona sin importar el orden de los datos, ejem. ordenando los datos de forma aleatoria:
  tstage4_shuffle = tstage4[0].orderBy(F.rand()) #df actual
  tstage4_expected_shuffle = tstage4_expected.orderBy(F.rand()) #df esperado
  assert tstage4_expected_shuffle.exceptAll(tstage4_shuffle).count() == 0
  assert tstage4_shuffle.exceptAll(tstage4_expected_shuffle).count() == 0
#prueba de almacenamiento
def test_stage5(tstage5):
  tstage5[0].show()
  assert type(tstage5) == list
  assert len(tstage5) == 1
  assert tstage5[0].count() == 32
#prueba completa con valores en cero
def test_zerodata(tzerodata, tzerodata_expected):
  tzerodata[0].show()
  tzerodata_expected.show()
  assert type(tzerodata) == list
  assert len(tzerodata) == 1
  assert tzerodata[0].count() == 32
  assert tzerodata[0].select('provincia').distinct().count() == 7
  assert tzerodata_expected.exceptAll(tzerodata[0]).count() == 0
  assert tzerodata[0].exceptAll(tzerodata_expected).count() == 0
#prueba completa con valores numéricos faltantes
def test_missnumdata(tmissnumdata, tmissnumdata_expected):
  tmissnumdata[0].show()
  tmissnumdata_expected.show()
  assert type(tmissnumdata) == list
  assert len(tmissnumdata) == 1
  assert tmissnumdata[0].count() == 32
  assert tmissnumdata[0].select('provincia').distinct().count() == 7
  assert tmissnumdata_expected.exceptAll(tmissnumdata[0]).count() == 0
  assert tmissnumdata[0].exceptAll(tmissnumdata_expected).count() == 0
#prueba completa con valores categóricos faltantes
def test_misscatdata(tmisscatdata, tmisscatdata_expected):
  tmisscatdata[0].show()
  tmisscatdata_expected.show()
  assert type(tmisscatdata) == list
  assert len(tmisscatdata) == 1
  assert tmisscatdata[0].count() == 32
  assert tmisscatdata[0].select('provincia').distinct().count() == 7
  assert tmisscatdata_expected.exceptAll(tmisscatdata[0]).count() == 0
  assert tmisscatdata[0].exceptAll(tmisscatdata_expected).count() == 0

Writing test_programaestudiante.py


In [8]:
#@title spark-submit programaestudiante.py { vertical-output: true, form-width: "50%", display-mode: "both" }
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 600})'''))
!spark-submit programaestudiante.py ciclista.csv ruta.csv actividad.csv
!python -m pytest -vv


<IPython.core.display.Javascript object>

21/08/08 19:33:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/08/08 19:33:42 INFO SparkContext: Running Spark version 3.1.2
21/08/08 19:33:42 INFO ResourceUtils: ==============================================================
21/08/08 19:33:42 INFO ResourceUtils: No custom resources configured for spark.driver.
21/08/08 19:33:42 INFO ResourceUtils: ==============================================================
21/08/08 19:33:42 INFO SparkContext: Submitted application: App#1
21/08/08 19:33:42 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
21/08/08 19:33:42 INFO ResourceProfile

---